# 2024 CITS4012 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

## imports

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger_eng")

stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


## Dataset Path




In [3]:
db_path = "dataset"

train_file = f"{db_path}/train.json"
test_file = f"{db_path}/test.json"
val_file = f"{db_path}/train.json"

# 1.Dataset Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [ ]:
import json
with open(train_file, "r") as fp:
    train = json.load(fp)
with open(val_file, "r") as fp:
    val = json.load(fp)
with open(test_file, "r") as fp:
    test = json.load(fp)

train['premise']['0'], train['hypothesis']['0'], train['label']['0'], len(train['premise'])

('The History of Polar Front and Air Mass Concept in the United States -\tThe best explanation for how air masses move across the united states is that the prevailing westerlies move air masses from west to east across the united states but may be deflected by the jet stream.\tneutral\tair masses<>move<>across the united states$$$the prevailing westerlies<>may be deflected<>by the jet stream$$$The best explanation for how air masses move across the united states<>is<>that the prevailing westerlies move air masses from west to east across the united states but may be deflected by the jet stream$$$the prevailing westerlies<>move<>air masses<>from west<>to east<>across the united states\nWhat is the name of the closes star to the Earth, other then the Sun?\tFrom earth, the sun appears brighter than any other star because the sun is the closest star\tneutral\tthe sun<>appears brighter<>because the sun is the closest star$$$the sun<>is<>the closest star\nWe call the year a shanah, because t

In [54]:
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk import pos_tag
import numpy as np


class Config:
    stem = False
    stemmer = PorterStemmer()
    lemmatize = False
    lemmatizer = WordNetLemmatizer()
    tokenize = True

    clean_text = True
    regex = r"[^a-z0-9\-\s]"

    pos = False
    pos_tagger = pos_tag

    lower = True

    stopwords = set(stopwords.words('english'))


class textProcesser():
    def __init__(self, cfg: Config) -> None:
        self.cfg = cfg

        self.lemmatizer = cfg.lemmatizer
        self.stemmer = cfg.stemmer
        self.stop_words = cfg.stopwords


    def clean(self, sentence: str) -> list[str]:
        tokens = [sentence]
        if self.cfg.tokenize:
            tokens = nltk.word_tokenize(sentence)

        if self.cfg.lemmatize:
            tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        if self.cfg.stem:
            tokens = [self.stemmer.stem(token) for token in tokens]

        if self.cfg.clean_text:
                
            tokens = [re.sub(self.cfg.regex, '', token.lower() if self.cfg.lower else token) for token in tokens]
            tokens = [token for token in tokens if token and token not in self.stop_words]

        if self.cfg.pos:
            tokens = self.cfg.pos_tagger(tokens)
        
        return tokens
    
    def clean_text(self, json):
        clean = {'premise': {}, 'hypothesis': {}, 'label': {}}
        for idx, key in enumerate(json['label'].keys()):
            if key == '0':
                print(json['label'][key])
                print(type(json['label'][key])) 
                print('neutral' in json['label'][key])
            if 'neutral' in json['label'][key]:
                clean['label'][idx] = 0
            else:
                clean['label'][idx] = 1

            clean['premise'][idx] = self.clean(json['premise'][key])
            clean['hypothesis'][idx] = self.clean(json['hypothesis'][key])
            
        return clean

In [55]:
clean_train = textProcesser(Config()).clean_text(train)
clean_val = textProcesser(Config()).clean_text(val)
clean_test = textProcesser(Config()).clean_text(test)

neutral
<class 'str'>
True
neutral
<class 'str'>
True
neutral
<class 'str'>
True


In [62]:
clean_train['premise'][2], clean_train['hypothesis'][2], clean_train['label'][2]


(['geysers', '-', 'periodic', 'gush', 'hot', 'water', 'surface', 'earth'],
 ['surface', 'sun', 'much', 'hotter', 'almost', 'anything', 'earth'],
 0)

In [63]:
from gensim.models import Word2Vec
sentences = list(clean_train['premise'].values()) + list(clean_train['hypothesis'].values())

model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=3, workers=2, sg=1)


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*